# Workshop 5: Airline Passenger Predictions with RNN (Felix code in PyTorch)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt

In [ ]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv

In [ ]:
# Load and plot the dataset
dataset_raw = pd.read_csv('airline-passengers.csv', usecols=[1], engine='python').values
plt.plot(dataset_raw)
plt.show()

In [ ]:
# Create dataset function
def create_dataset(dataset, look_back_memory=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back_memory-1):
        dataX.append(dataset[i:i+look_back_memory, 0])
        dataY.append(dataset[i+look_back_memory, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
# Preprocess the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset_raw.astype('float32'))
dataset.min(), dataset.max()

In [ ]:
# Split the dataset into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size], dataset[train_size:len(dataset)]
train.shape, test.shape

In [ ]:
# Create input and output sequences for training and testing
look_back_memory = 1
variables = 1
trainX, trainY = create_dataset(train, look_back_memory)
testX, testY = create_dataset(test, look_back_memory)
trainX.shape, trainY.shape, testX.shape, testY.shape

In [ ]:
# Reshape the input sequences for PyTorch
trainX = torch.from_numpy(trainX).unsqueeze(2).float()
trainY = torch.from_numpy(trainY).unsqueeze(1).float()
testX = torch.from_numpy(testX).unsqueeze(2).float()
testY = torch.from_numpy(testY).unsqueeze(1).float()
trainX.shape, trainY.shape, testX.shape, testY.shape

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(1234)

In [ ]:
# Create and train the model
model = SimpleRNN(variables, 10, variables)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
losses_train = []
losses_test = []
for epoch in range(100):
    optimizer.zero_grad()
    output = model(trainX)
    loss = criterion(output, trainY)
    loss.backward()
    optimizer.step()
    with torch.no_grad():
      output = model(testX)
      loss_test = criterion(output, testY)
    losses_train.append(loss.detach().numpy())
    losses_test.append(loss_test.detach().numpy())
    print("Loss train: " + str(losses_train[-1]) + " Loss test: " + str(losses_test[-1]))

In [ ]:
plt.plot(losses_train)
plt.plot(losses_test)
plt.xlabel("Epochs")
plt.ylabel("MSE")

In [ ]:
# Make predictions on the training and test sets
trainPredict = model(trainX).detach().numpy()
testPredict = model(testX).detach().numpy()

In [ ]:
# Rescale the predictions back to the original scale
trainPredict = scaler.inverse_transform(trainPredict)
trainY_orig = scaler.inverse_transform(trainY.numpy())
testPredict = scaler.inverse_transform(testPredict)
testY_orig = scaler.inverse_transform(testY.numpy())

In [ ]:
# Calculate root mean squared error (RMSE)
trainScore = math.sqrt(mean_squared_error(trainY_orig.flatten(), trainPredict.flatten()))
print('Train Score: %.2f MSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY_orig.flatten(), testPredict.flatten()))
print('Test Score: %.2f MSE' % (testScore))

In [ ]:
# Create empty plots for visualization
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back_memory:len(trainPredict)+look_back_memory, :] = trainPredict
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back_memory*2)+1:len(dataset)-1, :] = testPredict

In [ ]:
# Visualize the original data, training, and test predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
# Create and train the model
model = LSTMModel(variables, 10, variables)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
losses_train = []
losses_test = []
for epoch in range(100):
    optimizer.zero_grad()
    output = model(trainX)
    loss = criterion(output, trainY)
    loss.backward()
    optimizer.step()
    with torch.no_grad():
      output = model(testX)
      loss_test = criterion(output, testY)
    losses_train.append(loss.detach().numpy())
    losses_test.append(loss_test.detach().numpy())
    print("Loss train: " + str(losses_train[-1]) + " Loss test: " + str(losses_test[-1]))

In [ ]:
plt.plot(losses_train)
plt.plot(losses_test)
plt.xlabel("Epochs")
plt.ylabel("MSE")

In [ ]:
# Make predictions on the training and test sets
trainPredict = model(trainX).detach().numpy()
testPredict = model(testX).detach().numpy()

In [ ]:
# Rescale the predictions back to the original scale
trainPredict = scaler.inverse_transform(trainPredict)
trainY_orig = scaler.inverse_transform(trainY.numpy())
testPredict = scaler.inverse_transform(testPredict)
testY_orig = scaler.inverse_transform(testY.numpy())

In [ ]:
# Calculate root mean squared error (RMSE)
trainScore = math.sqrt(mean_squared_error(trainY_orig.flatten(), trainPredict.flatten()))
print('Train Score: %.2f MSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY_orig.flatten(), testPredict.flatten()))
print('Test Score: %.2f MSE' % (testScore))

In [ ]:
# Create empty plots for visualization
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back_memory:len(trainPredict)+look_back_memory, :] = trainPredict
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back_memory*2)+1:len(dataset)-1, :] = testPredict

In [ ]:
# Visualize the original data, training, and test predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
# Define the GRU model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
# Create and train the model
model = GRUModel(variables, 10, variables)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
losses_train = []
losses_test = []
for epoch in range(100):
    optimizer.zero_grad()
    output = model(trainX)
    loss = criterion(output, trainY)
    loss.backward()
    optimizer.step()
    with torch.no_grad():
      output = model(testX)
      loss_test = criterion(output, testY)
    losses_train.append(loss.detach().numpy())
    losses_test.append(loss_test.detach().numpy())
    print("Loss train: " + str(losses_train[-1]) + " Loss test: " + str(losses_test[-1]))

In [ ]:
plt.plot(losses_train)
plt.plot(losses_test)
plt.xlabel("Epochs")
plt.ylabel("MSE")

In [ ]:
# Make predictions on the training and test sets
trainPredict = model(trainX).detach().numpy()
testPredict = model(testX).detach().numpy()

In [ ]:
# Rescale the predictions back to the original scale
trainPredict = scaler.inverse_transform(trainPredict)
trainY_orig = scaler.inverse_transform(trainY.numpy())
testPredict = scaler.inverse_transform(testPredict)
testY_orig = scaler.inverse_transform(testY.numpy())

In [ ]:
# Calculate root mean squared error (RMSE)
trainScore = math.sqrt(mean_squared_error(trainY_orig.flatten(), trainPredict.flatten()))
print('Train Score: %.2f MSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY_orig.flatten(), testPredict.flatten()))
print('Test Score: %.2f MSE' % (testScore))

In [ ]:
# Create empty plots for visualization
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back_memory:len(trainPredict)+look_back_memory, :] = trainPredict
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back_memory*2)+1:len(dataset)-1, :] = testPredict

In [ ]:
# Visualize the original data, training, and test predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()